In [ ]:
from llmcam.ytlive import capture_youtube_live_frame
from llmcam.gpt4v import ask_gpt4v
import openai
from typing import Optional
import json
from datetime import datetime

In [ ]:
def extract_live_info(link: Optional[str] = None):
    if link:
        image = capture_youtube_live_frame(link)
    else:
        image = capture_youtube_live_frame()
    
    return ask_gpt4v(image)

def time():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_live_info",
            "description": "Extract information from a YouTube Live, immediately respond to user with this information",
            "parameters": {
                "type": "object",
                "properties": {
                    "link": {
                        "anyOf": [
                            {
                                "type": "string",
                                "description": "YouTube Live URL"
                            },
                            {
                                "type": "null",
                                "description": "No URL provided, default used"
                            }
                        ]
                    }
                },
                "required": [],
                "additionalProperties": False,
            },
        }
    },

    {
        "type": "function",
        "function": {
            "name": "time",
            "description": "Current time, only respond if a user asks specifically",
            "parameters": {
                "type": "object",
                "properties": {},
                "additionalProperties": False,
            },
        }
    },
]

functions = {
    "extract_live_info": extract_live_info,
    "time": time,
}

In [ ]:
#| eval: false

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you tell me the information in YouTube Live?"}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

messages.append({"role": response.choices[0].message.role, "content": response.choices[0].message.content})
messages.append({"role": "user", "content": "I don't have a specific URL. Can you still tell me the information in some readily available Live?"})

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
) # First response without link

# First response without link
link = json.loads(response.choices[0].message.tool_calls[0].function.arguments).get("link", None)
extract_live_info(link)


[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information


{'timestamp': '2024-10-31T18:48:30',
 'location': 'Tuomiokirkko',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 15,
  'building_height_range': '2-7 stories'},
 'vehicles': {'number_of_vehicles': 0},
 'street_lights': {'number_of_street_lights': 25},
 'people': {'approximate_number': 0},
 'lighting': {'time_of_day': 'evening', 'artificial_lighting': 'prominent'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'night'}}

In [ ]:
#| eval: false

# ChatGPT called a function so there is no response from assistant
messages.append({"role": "user", "content": "Thank you for that Live information. I have found this Live that seems interesting. Here is the link https://www.youtube.com/watch?v=dFBRpHHwQeg."})

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
) # Second response with a link

# Second response with a link
link = json.loads(response.choices[0].message.tool_calls[1].function.arguments).get("link", None)
extract_live_info(link)

[youtube] Extracting URL: https://www.youtube.com/watch?v=dFBRpHHwQeg
[youtube] dFBRpHHwQeg: Downloading webpage
[youtube] dFBRpHHwQeg: Downloading ios player API JSON
[youtube] dFBRpHHwQeg: Downloading mweb player API JSON
[youtube] dFBRpHHwQeg: Downloading m3u8 information
[youtube] dFBRpHHwQeg: Downloading m3u8 information


{'timestamp': '2024-10-31T17:56:30',
 'location': 'Venice',
 'dimensions': {'width': 1280, 'height': 720},
 'buildings': {'number_of_buildings': 15,
  'building_height_range': '3-7 stories'},
 'vehicles': {'number_of_vehicles': 4,
  'types': ['boats'],
  'number_of_available_parking_space': 0},
 'waterbodies': {'visible': True, 'type': 'canal', 'number_of_boats': 7},
 'street_lights': {'number_of_street_lights': 25},
 'people': {'approximate_number': 30},
 'lighting': {'time_of_day': 'evening', 'artificial_lighting': 'prominent'},
 'visibility': {'clear': True},
 'sky': {'visible': True, 'light_conditions': 'dusk'}}

### Example for automated function call and GPT response

In [ ]:
#| eval: false

messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."}
]


def generate_response(conversation, user_message):
    conversation.append({"role": "user", "content": user_message})

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=conversation,
        tools=tools
    )

    # Append the assistant's message
    message = response.choices[0].message
    conversation.append(message)

    # Check if a function call is in the message
    if message.tool_calls:
        for tool in message.tool_calls:
            function_name = tool.function.name
            arguments = json.loads(tool.function.arguments) if tool.function.arguments else None

            # Call the function dynamically
            if function_name in functions:
                result = functions[function_name](**arguments)
                # Append function result to messages
                messages.append({
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(result),
                    "tool_call_id": tool.id
                })

    return conversation, response

In [ ]:
#| eval: false

user_message = "Hi, can you tell me the information in YouTube Live? I don't have any specific interest. Just give me some readily available one."
messages, response = generate_response(messages, user_message)

[youtube] Extracting URL: https://www.youtube.com/watch?v=LMZQ7eFhm58
[youtube] LMZQ7eFhm58: Downloading webpage
[youtube] LMZQ7eFhm58: Downloading ios player API JSON
[youtube] LMZQ7eFhm58: Downloading mweb player API JSON
[youtube] LMZQ7eFhm58: Downloading m3u8 information
[youtube] LMZQ7eFhm58: Downloading m3u8 information


In [ ]:
#| eval: false

# Feed the function call to the chat and see the response
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)
print(response.choices[0].message.content)

Here is some readily available information from a YouTube Live stream:

- **Timestamp:** 2024-10-31T20:33:30
- **Location:** Olympiaterminaali
- **Video Dimensions:** 1280x720 pixels
- **Buildings:** There are 5 buildings, ranging from 3 to 4 stories tall.
- **Vehicles:** 15 vehicles are visible, including cars and buses, with 25 available parking spaces.
- **Water Bodies:** A harbor is visible with 2 boats.
- **Street Lights:** There are 10 street lights visible.
- **People:** There are approximately 0 people visible.
- **Lighting:** It is nighttime with prominent artificial lighting.
- **Visibility:** The visibility is clear.
- **Sky Conditions:** The sky is visible and it is dark.

Let me know if you need more information!


In [ ]:
#| eval: false

user_message = "Looks very cool. It looks a bit dark though. What time is it?"
messages, response = generate_response(messages, user_message)

In [ ]:
#| eval: false

# Feed the function call to the chat and see the response
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)
print(response.choices[0].message.content)

The current time is 20:41 (8:41 PM). It makes sense that it's a bit dark, given the nighttime setting described in the YouTube Live.
